# Uporaba indeksov v bazah

Če želimo delati hitre poizvedbe po določenih stolpcih, lahko na njih uvedemo **indekse**.

In [ ]:
import sqlite3
from urllib.request import urlretrieve

Poglejmo si dve bazi, ki vsebujeta podatke o nekaj več kot 200000 grafih iz projekta [*discrete*ZOO](https://discretezoo.xyz/). Prva vsebuje indekse in je velika 426 MB, druga pa vsebuje iste podatke, a so bili indeksi odstranjeni (z izjemo ključev in `UNIQUE`), tako da je njena velikost 385 MB.

In [ ]:
_ = urlretrieve("http://baza.fmf.uni-lj.si/discretezoo.db", "discretezoo.db")
_ = urlretrieve("http://baza.fmf.uni-lj.si/discretezoo-noindex.db", "discretezoo-noindex.db")

Vzpostavimo povezavi na obe bazi in napišimo funkcijo, ki bo primerjala trajanje izvajanja iste poizvedbe na vsaki od baz.

In [ ]:
c1 = sqlite3.connect("discretezoo.db")
c2 = sqlite3.connect("discretezoo-noindex.db")

In [ ]:
def primerjaj(*largs):
    %timeit c1.execute(*largs).fetchall()
    %timeit c2.execute(*largs).fetchall()

Poglejmo si, kateri indeksi obstajajo v vsaki bazi.

In [ ]:
c1.execute("SELECT sql FROM sqlite_master WHERE type = 'index'").fetchall()

In [ ]:
c2.execute("SELECT sql FROM sqlite_master WHERE type = 'index'").fetchall()

Poskusimo sedaj prvo poizvedbo - štetje vrstic v tabeli `graph` z določeno vrednostjo v stolpcu `order`.

In [ ]:
sql = """
    SELECT COUNT(*) FROM graph WHERE `order` = ?
"""
primerjaj(sql, [512])

Opazimo, da je poizvedba na bazi z indeksi za nekaj velikostnih razredov hitrejša kot na bazi brez indeksov. SQLite si sicer poizvedbo zapomni in ustvari začasen indeks, zaradi česar so naslednje poizvedbe hitrejše. Poskusimo sedaj z branjem celotnih vrstic.

In [ ]:
sql = """
    SELECT * FROM graph WHERE `order` = ?
"""
primerjaj(sql, [512])

Razlika je še vedno očitna. Poskusimo še z združevanjem.

In [ ]:
sql = """
    SELECT `order`, AVG(diameter) FROM graph GROUP BY `order`
"""
primerjaj(sql)